In [109]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [110]:
def generate_data(num_rows=1000):
    np.random.seed(0)
    years = np.random.choice(range(2000, 2024), num_rows)
    countries = np.random.choice(['Germany', 'France', 'Italy', 'Spain', 'UK', 'Belgium', 'Netherlands', 'Denmark', 'Sweden', 'Norway', 'Finland'], num_rows)
    construction_values = np.random.uniform(1000, 50000, num_rows)
    labor_wages = np.random.uniform(10, 50, num_rows)
    building_costs = np.random.uniform(500, 30000, num_rows)

    return pd.DataFrame({'Year': years, 'Country': countries,
                         'Construction Value': construction_values,
                         'Labor Wages': labor_wages,
                         'Building Cost': building_costs})

In [111]:
df = generate_data()

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

In [112]:
@app.callback(
    Output('scatter-plot', 'figure'),
    Input('country-dropdown', 'value')
)
def update_scatter_plot(countries):
    df_filtered = df[df['Country'].isin(countries)]
    scatter_plot = go.Scatter(x=df_filtered['Year'],
                              y=df_filtered['Construction Value'],
                              mode='markers',
                              marker=dict(size=10, color='#1E88E5'))
    layout = go.Layout(title='Construction Value over Time',
                       xaxis=dict(title='Year'),
                       yaxis=dict(title='Construction Value'),
                       font=dict(family="Lato"),
                       plot_bgcolor="#242424",
                       paper_bgcolor="#242424",
                       font_color="#FFFFFF")
    return {'data': [scatter_plot], 'layout': layout}

In [113]:
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('year-slider', 'value'), Input('top-10-check', 'value')]
)
def update_bar_chart(year, top10):
    df_filtered = df[df['Year'] == year]
    if 'TOP10' in top10:
        df_filtered = df_filtered.groupby('Country')['Construction Value'].mean().nlargest(10).reset_index()
    else:
        df_filtered = df_filtered.groupby('Country')['Construction Value'].mean().reset_index()

    bar_chart = go.Bar(x=df_filtered['Country'],
                       y=df_filtered['Construction Value'],
                       marker=dict(color='#FFC107'))
    layout = go.Layout(title=f'Construction Value by Country in {year}',
                       xaxis=dict(title='Country'),
                       yaxis=dict(title='Construction Value'),
                       font=dict(family="Lato"),
                       plot_bgcolor="#242424",
                       paper_bgcolor="#242424",
                       font_color="#FFFFFF")
    return {'data': [bar_chart], 'layout': layout}

In [114]:
@app.callback(
    Output('histogram', 'figure'),
    Input('country-dropdown', 'value')
)
def update_histogram(countries):
    df_filtered = df[df['Country'].isin(countries)]
    histogram = go.Histogram(x=df_filtered['Labor Wages'],
                             nbinsx=20,
                             marker=dict(color='#00796B'))
    layout = go.Layout(title='Distribution of Labor Wages',
                       xaxis=dict(title='Labor Wage'),
                       yaxis=dict(title='Frequency'),
                       font=dict(family="Lato"),
                       plot_bgcolor="#242424",
                       paper_bgcolor="#242424",
                       font_color="#FFFFFF")
    return {'data': [histogram], 'layout': layout}

In [115]:
@app.callback(
    Output('pie-chart', 'figure'),
    Input('year-slider', 'value')
)
def update_pie_chart(year):
    df_filtered = df[df['Year'] == year]
    df_grouped = df_filtered.groupby('Country')['Construction Value'].mean()

    pie_chart = go.Pie(labels=df_grouped.index, values=df_grouped.values)
    layout = go.Layout(title=f'Average Construction Value by Country in {year}',
                       font=dict(family="Lato"),
                       plot_bgcolor="#242424",
                       paper_bgcolor="#242424",
                       font_color="#FFFFFF")
    return {'data': [pie_chart], 'layout': layout}

In [116]:
app.layout = html.Div(children=[
    html.H1("Construction Dashboard",
            style={'text-align':'center', 'padding':'10px', 'color':'#FFFFFF', 'font-family':'Lato'}),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='country-dropdown',
                options=[
                    {'label': country, 'value': country}
                    for country in df['Country'].unique()
                ],
                value=df['Country'].unique().tolist(),
                multi=True
            ),

            dcc.Graph(id='scatter-plot'),
        ], md=6),

        dbc.Col([
            dbc.Checklist(
                id='top-10-check',
                options=[{'label': 'Top 10 Countries Only', 'value': 'TOP10'}],
                value=[]
            ),

            dcc.Graph(id='bar-chart'),
        ], md=6),
    ]),

    dbc.Row([
        dbc.Col([
            dcc.Slider(
                id='year-slider',
                min=df['Year'].min(),
                max=df['Year'].max(),
                value=df['Year'].min(),
                marks={str(year): str(year) for year in df['Year'].unique()},
                step=None
            ),

            dcc.Graph(id='pie-chart'),
        ], md=6),

        dbc.Col([
            dcc.RadioItems(
                id='wage-cost-radio',
                options=[
                    {'label': 'Labor Wages', 'value': 'Labor Wages'},
                    {'label': 'Building Cost', 'value': 'Building Cost'},
                ],
                value='Labor Wages'
            ),

            dcc.Graph(id='histogram'),
        ], md=6),
    ]),
])

Finally run the app, it will run on your local network, e.g. http://127.0.0.1:XXXX/

In [ ]:
app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [26/May/2023 14:41:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_favicon.ico?v=2.10.0 HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [26/May/2023 14:41:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/May/2023 14:41:54] "POST /_dash-updat